In [1]:
import absbox 

#Hastructure version ✅Connected, local lib:0.27.24, server:0.27.6

In [6]:
receivable1 = ["Invoice"
              ,{"start":"2024-04-01","originBalance":2000
                 ,"originAdvance":1500,"dueDate":"2024-08-01"
                ,"feeType":("Fixed",150)}
              ,{"status":"Current"}]
receivable4 = ["Invoice"
              ,{"start":"2024-04-01","originBalance":2000
                 ,"originAdvance":1500,"dueDate":"2024-06-01"
                ,"feeType":("FactorFee", 0.01, 25, "Up")}
              ,{"status":"Current"}]
pool = {'assets':[receivable1,receivable4,receivable4,receivable1,receivable4,receivable1],'issuanceStat':{"IssuanceBalance":100000}}

dates={"collect": ["2024-02-29","2024-03-31"]
        ,"pay": ["2024-02-15"  
                      ,"2024-03-15"]  
        ,"stated":"2027-11-04"
        ,"poolFreq":"MonthFirst"
        ,"payFreq":["DayOfMonth",15]
        }

accounts={"collections":{"balance":50_000_000},
            "revolBuyAcc":{"balance":100_000}
            ,"purchaserAccount": {"balance":0}
            ,"sellerAccount": {"balance": 0}
          ,"cashReserve":{"balance":0}}
          
bonds={"A1":{"balance":150_000_000
                    ,"rate":0.15
                    ,"originBalance":200_000_000
                    ,"originRate":0.0858
                    ,"startDate":"2022-11-04"
                    ,"rateType":{"fix": 0.15, "dayCount":"DC_ACT_365A"}
                    ,"maturityDate":"2027-11-04"
                    ,"bondType":{"Lockout":"2023-11-04"}}
                    ,"B":{
                    "balance":1
                    ,"rate":0.0
                    ,"originBalance":1000
                    ,"originRate":0.0
                    ,"startDate":"2022-11-04"
                    ,"rateType":{"Fixed":0.00}
                    ,"bondType":{"Equity":None}
                    }
                    }

fees={"technology_fee":{"type":{"annualPctFee":[("bondBalance",),0.004]}}}

waterfall = {"Revolving": [
              ["payFee","collections",["technology_fee"]]
              ,["accrueAndPayInt","collections",["A1"]]
              ,["transfer" ,"collections","revolBuyAcc"]
              ,["buyAsset",["Current|Defaulted",1.0,0],"revolBuyAcc",None] #This line is the problematic one. If I remove it the server will run the deal
              ,["payIntResidual","collections","B"]]
  
  ,"endOfCollection":[["calcFee","technology_fee"] ,["calcInt","A1"]]

        ,"cleanUp":[["sellAsset", ["Current|Defaulted",1.0,0], "collections"] #I also comment out this line as well
                    ,["transfer" ,"cashReserve",'collections']
                    ,["transfer" ,"revolBuyAcc",'collections']
                    ,["accrueAndPayInt","collections",["A1"]]
                    ,["payPrin","collections",["A1"]]
                    ,["payIntResidual","collections","B"]]}
        
collects=[#["CollectedCash","collections"]
          ["CollectedPrincipal","collections"]
          ,["CollectedFeePaid","collections"]]

trigger=None
deal_data = {
          "name":"Deal"
          ,"dates":dates
          ,"pool":pool
          ,"accounts":accounts
          ,"fees":fees
          ,"bonds":bonds
          ,"waterfall":waterfall
          ,"collect":collects
          ,"trigger":trigger
          ,"status":"Revolving"
        } 

from absbox import mkDeal, API
deal=mkDeal(deal_data)


revol_assets = [["Invoice",
                  {
                    "start":"2024-05-01",
                    "originBalance":12000,
                    "originAdvance":10000,
                    "dueDate":"2024-08-01",
                    "feeType":("FixedRate",0.132)
                }
                  ,{
                    "status":"current"
                    }]
                ,["Invoice",
                  {
                    "start":"2024-05-01",
                    "originBalance":12000,
                    "originAdvance":10000,
                    "dueDate":"2024-09-01",
                    "feeType":("FixedRate",0.132)
                }
                  ,{
                    "status":"current"
                    }]
               ]
revolving=("revolving"
        ,["constant",*revol_assets]
        ,("Pool",("Receivable",{"CDR":0.5},None,None),None,None))
      
#localAPI = API("https://absbox.org/api/dev",lang='english')
localAPI = API("http://localhost:8081",lang='english',check=False)

r = localAPI.run(deal
              ,poolAssump = ("Pool",("Receivable",{"CDR":0.5},None,None),None,None)
              ,runAssump = [("inspect"
                                ,("MonthEnd",("status",("Amortizing")))
                                ,("MonthEnd",("accountBalance",))
                                ,("MonthEnd",("poolBalance",))
                                ,("MonthEnd",("bondBalance",))
                                )
                            ,revolving
                            ,("call",{"afterDate":"2025-11-04"})
                            #,("stop","2024-05-30")
                          ]
            ,read=True)

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.26.4, server:0.27.18

Warning Message from server:
Bond A1 is not paid off
Bond B is not paid off
Fee technology_fee is not paid off

In [7]:
r['pool']['flow']

,Balance,AccuredFee,Principal,Fee,Default,Recovery,Loss,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,
2024-03-31,48329213.12,0,0.00,0.00,0.00,0,0.00,0,0,0,0.00,0,0.00
2024-04-01,48329213.12,0,0.00,0.00,0.00,0,0.00,0,0,0,0.00,0,0.00
2024-05-01,48329213.12,0,0.00,0.00,0.00,0,0.00,0,0,0,0.00,0,0.00
2024-06-01,48323213.12,0,5183.43,160.29,656.28,0,656.28,0,0,0,656.28,0,656.28
2024-07-01,24164606.56,0,17608223.94,2677748.33,3872634.29,0,3872634.29,0,0,0,3873290.57,0,3873290.57
2024-08-01,12378270.34,0,16605951.68,2525016.48,5033638.40,0,5033638.40,0,0,0,8906928.97,0,8906928.97
2024-09-01,29541291.42,0,0.00,0.00,0.00,0,0.00,0,0,0,8906928.97,0,8906928.97
2024-10-01,29541291.42,0,0.00,0.00,0.00,0,0.00,0,0,0,8906928.97,0,8906928.97
2024-11-01,23352156.25,0,4511008.45,686005.88,992120.84,0,992120.84,0,0,0,9899049.81,0,9899049.81


In [92]:
r['accounts']['revolBuyAcc']

,balance,change,memo
date,,,
2024-03-15,48317213.12,48217213.12,"<Transfer:collections,revolBuyAcc>"
2024-03-15,0.00,-48317213.12,<PurchaseAsset:>
2024-04-15,0.00,0.00,"<Transfer:collections,revolBuyAcc>"
2024-04-15,0.00,0.00,<PurchaseAsset:>
2024-05-15,0.00,0.00,"<Transfer:collections,revolBuyAcc>"
2024-05-15,0.00,0.00,<PurchaseAsset:>
2024-06-15,0.00,0.00,"<Transfer:collections,revolBuyAcc>"
2024-06-15,0.00,0.00,<PurchaseAsset:>
2024-07-15,40406167.48,40406167.48,"<Transfer:collections,revolBuyAcc>"


In [93]:
import pandas
pandas.set_option('display.max_rows', None)


In [94]:
r['accounts']['collections']

,balance,change,memo
date,,,
2024-03-15,50000000.00,0.00,<SeqPayFee:technology_fee>
2024-03-15,48217213.12,-1782786.88,<PayInt:A1>
2024-03-15,0.00,-48217213.12,"<Transfer:collections,revolBuyAcc>"
2024-03-15,0.00,0.00,<PayYield:B>
2024-03-31,0.00,0.00,<Pool:CollectedFeePaid>
2024-03-31,0.00,0.00,<Pool:CollectedPrincipal>
2024-04-01,0.00,0.00,<Pool:CollectedFeePaid>
2024-04-01,0.00,0.00,<Pool:CollectedPrincipal>
2024-04-15,0.00,0.00,<SeqPayFee:technology_fee>


In [95]:
r['result']['status']

,Date,From,To
0,2025-11-15,Revolving,Called


In [96]:
r['pool']['flow']

,Balance,AccuredFee,Principal,Fee,Default,Recovery,Loss,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,
2024-03-31,48321213.12,0,0.00,0.00,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2024-04-01,48321213.12,0,0.00,0.00,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2024-05-01,48321213.12,0,0.00,0.00,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2024-06-01,48319213.12,0,1940.00,60.00,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2024-07-01,2000.00,0,44983325.42,3333887.70,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2024-08-01,40406167.48,0,1850.00,150.00,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2024-09-01,40406167.48,0,0.00,0.00,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2024-10-01,40406167.48,0,0.00,0.00,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2024-11-01,0.00,0,37618141.93,2788025.55,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
